# Task 12 - CBC-MAC CCM
Krypto Lab

Felix Kleinsteuber, Matrikelnummer: 185 709

CBC-MAC ist ein Message Authentication Code (MAC), der auf den Cipher-Block-Chaining-Betriebsmodus (CBC) einer symmetrischen Blockchiffre (hier: AES) zurückgreift. Der Schlüssel des MAC ist auch der Schlüssel, der für AES verwendet wird. Die Ausgabe des letzten Blocks ist dann der Hashwert.

In [12]:
import re
import numpy as np
import random

## 1. AES Encrypt/Decrypt (Task 3)

In [13]:
def read_sbox_file(filename):
    with open(filename, "r") as f:
        content = f.read()
        # Werte in einer Liste der Länge 256 voneinander isolieren
        vals = re.split(",\s", content)
        # Werte als Hexadezimalzahlen parsen
        int_vals = [int(val, 16) for val in vals]
        # Zu numpy Array konvertieren (ein Wert = 1 byte = np.uint8) und reshape
        return np.array(int_vals, dtype=np.uint8).reshape((16, 16))

sbox = read_sbox_file("SBox.txt")
assert sbox[0,0] == 0x63
sbox_inv = read_sbox_file("SBoxInvers.txt")
assert sbox_inv[0,0] == 0x52

def str_to_matrix(str):
    str += chr(0) * (16 - len(str))
    return np.array([ord(c) for c in str], dtype=np.uint8).reshape((4,4)).transpose()

def matrix_to_str(mat):
    chars = [chr(c) for c in np.nditer(mat.transpose(), order="C")]
    return "".join(chars)

# Test
input_str = "Das ist ein Test"
mat = str_to_matrix(input_str)
out_str = matrix_to_str(mat)
assert input_str == out_str

def add_round_key(mat, key):
    return np.bitwise_xor(mat, key)

# Test
test_mat = np.array(mat, dtype=np.uint8)
test_key = str_to_matrix("1234567890123456")
test_mat_key = add_round_key(test_mat, test_key)
test_mat_key_inv = add_round_key(test_mat_key, test_key)
assert np.all(test_mat == test_mat_key_inv)

# Zugriff auf S-Box für einzelnes Byte
def get_sbox(val, inv):
    s_row = (val & 0xf0) >> 4
    s_col = val & 0xf
    return sbox_inv[s_row, s_col] if inv else sbox[s_row, s_col]

def sub_bytes(mat, inv=False):
    # appy get_sbox on every value in matrix
    f = np.vectorize(lambda val: get_sbox(val, inv))
    return f(mat)

# Test
test_mat = np.array(mat, dtype=np.uint8)
sub_mat = sub_bytes(test_mat)
subinv_mat = sub_bytes(sub_mat, inv=True)
assert np.all(test_mat == subinv_mat)

def shift_rows(mat, inv=False):
    if inv:
        # Zyklische Verschiebung nach rechts
        mat[1,:] = mat[1,[3,0,1,2]]
        mat[2,:] = mat[2,[2,3,0,1]]
        mat[3,:] = mat[3,[1,2,3,0]]
    else:
        # Zyklische Verschiebung nach links
        mat[1,:] = mat[1,[1,2,3,0]]
        mat[2,:] = mat[2,[2,3,0,1]]
        mat[3,:] = mat[3,[3,0,1,2]]
    return mat

# Test
test_mat = np.array(mat, dtype=np.uint8)
shifted_mat = shift_rows(test_mat)
shiftedinv_mat = shift_rows(shifted_mat, inv=True)
assert np.all(test_mat == shiftedinv_mat)

# gemaess Foliendefinition, multipliziert Polynom a(x) mit Polynom p(x) = x
def xtimes(a):
    t = a << 1
    if a & 0b10000000 != 0:
        t = np.bitwise_xor(t, 0x1b)
    return t

# Matrizen zur Multiplikation
mc_mat = np.array([
    [2, 3, 1, 1],
    [1, 2, 3, 1],
    [1, 1, 2, 3],
    [3, 1, 1, 2]], dtype=np.uint8)
mc_mat_inv = np.array([
    [0xe, 0xb, 0xd, 0x9],
    [0x9, 0xe, 0xb, 0xd],
    [0xd, 0x9, 0xe, 0xb],
    [0xb, 0xd, 0x9, 0xe]], dtype=np.uint8)

# Funktion zum Ausfuehren der Matrixmultiplikation auf einer einzelnen Spalte
def mix_single_col(in_col, inv):
    global mc_mat, mc_mat_inv
    mat = mc_mat_inv if inv else mc_mat
    out_col = np.zeros(in_col.shape, dtype=np.uint8)
    # Iteriere durch Matrix
    for row in range(4):
        for col in range(4):
            # Iteriere durch bits in mat[row, col]
            for bitpos in range(8):
                if mat[row, col] & (1 << bitpos) != 0:
                    # Multipliziere einstelliges Polynom (z.B. x^2 = x * x) mit Eingabepolynom
                    prod = in_col[col]
                    for _ in range(bitpos):
                        prod = xtimes(prod)
                    # Addiere das Produkt zum Ausgabepolynom
                    out_col[row] = np.bitwise_xor(out_col[row], prod)
    return out_col

def mix_columns(mat, inv=False):
    for col in range(4):
        mat[:,col] = mix_single_col(mat[:,col], inv)
    return mat

# Test
test_mat = np.array(mat, dtype=np.uint8)
mixed_test_mat = mix_columns(test_mat)
mixed_test_mat_inv = mix_columns(mixed_test_mat, inv=True)
assert np.all(test_mat == mixed_test_mat_inv)

def aes_encrypt(mat, keys, verbose=False):
    assert mat.shape == (4,4)
    assert keys.shape == (11, 4, 4)
    enc_mat = add_round_key(np.array(mat, dtype=np.uint8), keys[0])
    for i in range(1, 10):
        enc_mat = sub_bytes(enc_mat)
        enc_mat = shift_rows(enc_mat)
        enc_mat = mix_columns(enc_mat)
        enc_mat = add_round_key(enc_mat, keys[i])
        if verbose:
            print(i, matrix_to_str(enc_mat))
    enc_mat = sub_bytes(enc_mat)
    enc_mat = shift_rows(enc_mat)
    enc_mat = add_round_key(enc_mat, keys[10])
    return enc_mat

def aes_decrypt(mat, keys, verbose=False):
    assert mat.shape == (4,4)
    assert keys.shape == (11, 4, 4)
    # xor muss nicht umgekehrt werden (in x-or key = out <=> out x-or key = in)
    dec_mat = add_round_key(np.array(mat, dtype=np.uint8), keys[10])
    dec_mat = shift_rows(dec_mat, inv=True)
    dec_mat = sub_bytes(dec_mat, inv=True)
    for i in range(9, 0, -1):
        if verbose:
            print(i, matrix_to_str(enc_mat))
        dec_mat = add_round_key(dec_mat, keys[i])
        dec_mat = mix_columns(dec_mat, inv=True)
        dec_mat = shift_rows(dec_mat, inv=True)
        dec_mat = sub_bytes(dec_mat, inv=True)
    dec_mat = add_round_key(dec_mat, keys[0])
    return dec_mat

# Test
test_mat = np.array(mat, dtype=np.uint8)
keys = np.random.randint(0, 255, size=(11, 4, 4), dtype=np.uint8)
enc_test_mat = aes_encrypt(test_mat, keys)
dec_test_mat = aes_decrypt(enc_test_mat, keys)
assert np.all(dec_test_mat == test_mat)

## 2. Schlüsselgenerierung (Task 4)

In [14]:
# Falls word als einzelner int gegeben
def sub_word_alt(word):
    # Substituiere jedes Byte im Wort einzeln
    out = 0
    for i in range(4):
        b = (word >> (i * 8)) & 0xff
        s_b = get_sbox(b)
        out |= s_b << (i * 8)
    return out

# ab hier word als [b0, b1, b2, b3] gegeben
def sub_word(word):
    return sub_bytes(word)

def rot_word(word):
    return np.roll(word, shift=-1)

# Test
assert np.all(rot_word([0, 1, 2, 3]) == np.array([1, 2, 3, 0]))

def rcon(i):
    vec = np.zeros(4, dtype=np.uint8)
    vec[0] = 1
    for j in range(i - 1):
        vec[0] = xtimes(vec[0])
    return vec

# Test
rci = np.array([rcon(i) for i in range(1, 11)])
assert np.all(rci[:,0] == np.array([0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36]))
assert np.all(rci[:,1:] == 0)

def generate_round_keys(key):
    assert key.shape == (4,4)
    W = np.zeros((11, 4, 4), dtype=np.uint8)
    W[0] = key
    for i in range(4, 44):
        k = i // 4
        if i % 4 == 0:
            # W[i] = W[i-4] ^ rcon(i/4) ^ SubWord(RotWord(W[i-1]))
            W[k, :, 0] = W[k - 1, :, 0] ^ rcon(k) ^ sub_word(rot_word(W[k - 1, :, 3]))
        else:
            # W[i] = W[i-4] ^ W[i-1]
            W[k, :, i % 4] = W[k - 1, :, i % 4] ^ W[k, :, i % 4 - 1]
    return W

# Test
key_text = "EINSCHLUESSEL123"
# generate_round_keys(str_to_matrix(key_text))

## 3. CBC
Wir nutzen AES im Cipher Block Chaining (CBC) Modus. Die AES-Implementierung aus Task 3 lässt bloß die Verschlüsselung von Blockmatrizen (4x4 Bytes) zu. Wir benötigen daher zunächst eine Hilfsfunktion, die einen beliebigen unsigned integer byteweise in eine solche Matrix einträgt.

In [15]:
def int_to_matrix(n: int):
    assert 0 <= n < 2 ** 128
    return np.array([(n >> (8 * b)) & 0xFF for b in range(15, -1, -1)], dtype=np.uint8).reshape((4,4)).transpose()

# Test
mat_out = int_to_matrix(243 << 64 | 42 << 16 | 127)
print(mat_out)
assert mat_out[3,3] == 127
assert mat_out[1,3] == 42
assert mat_out[3,1] == 243

[[  0   0   0   0]
 [  0   0   0  42]
 [  0   0   0   0]
 [  0 243   0 127]]


CBC ver-xor-t jeweils den Kryptotext des letzten Blocks mit dem Klartext des nächsten. Der erste Block wird mit dem Initialisierungsvektor (hier: 0) ver-xor-t. Der letzte Kryptotext ist der gesuchte Hashwert.

Da AES aus dem Schlüssel die immer gleichen 11 Rundenschlüssel erzeugt, übergebe ich direkt die Rundenschlüssel, statt diese jedes Mal neu zu berechnen.

In [16]:
def aes_cbc_mac(text: str, round_keys):
    # Initialisierungsvektor = 0
    last_cipher = int_to_matrix(0)
    for i in range(0, len(text), 16):
        # plaintext
        block = str_to_matrix(text[i:(i+16)])
        # mit letztem Ciphertext verxoren
        block = np.bitwise_xor(block, last_cipher)
        # mit AES zu neuem Ciphertext verschlüsseln
        last_cipher = aes_encrypt(block, round_keys)
    return last_cipher


## 3. CCM-Modus
CCM = Counter with CBC-MAC.

Neben dem Hashwert (mit CBC-MAC) soll auch der Kryptotext selbst übertragen werden. Dazu nutzen wir den Counter-Betriebsmodus. Statt des Klartexts verschlüsseln wir pro Block einen 128-Bit-Counter, der pro Block um 1 erhöht wird. Die Ausgabe von AES ver-xor-en wir anschließend mit dem Klartext, um den Kryptotext zu erhalten. Als letzten Block hängen wir die Ausgabe von CBC-MAC ver-xor-t mit dem Initialwert des Counters an.

In [17]:
def generate_nonce():
    return random.getrandbits(64)

def aes_counter(text: str, initial_ctr: int, round_keys):
    # Encrypt: Im Counter-Modus mit AES verschlüsseln
    y = "" # Kryptotext
    for i in range(1, (len(text) - 1) // 16 + 2):
        # Für AES teile Klartext in m=128 Bit Blöcke
        # Berechne Folge T_i = ctr + i mod 2^m für i = 0, ..., n
        T_i = (initial_ctr + i) % (2 ** 128)
        enc_T_i = aes_encrypt(int_to_matrix(T_i), round_keys)
        x_i = str_to_matrix(text[(16*(i-1)):(16*i)])
        # y_i = x_i ^ E(k, T_i)
        y_i = np.bitwise_xor(enc_T_i, x_i)
        y += matrix_to_str(y_i)
    return y

def ccm_encrypt_and_hash(text: str, nonce: int, key: str = "EINSCHLUESSEL123"):
    round_keys = generate_round_keys(str_to_matrix(key))
    print(round_keys)
    ctr = nonce << 64
    # Encrypt mit AES im Counter Mode
    y = aes_counter(text, ctr, round_keys)
    # Hash: Füge y' an (CBC)
    tmp = aes_cbc_mac(text, round_keys)
    y_ = np.bitwise_xor(int_to_matrix(ctr), tmp)
    y += matrix_to_str(y_)
    return y

# Test
my_text = "Hallo! Wir testen heute das Counter with CBC-MAC Verfahren."
nonce = generate_nonce()
enc = ccm_encrypt_and_hash(my_text, nonce)
print(enc, nonce)

[[[ 69  67  69  76]
  [ 73  72  83  49]
  [ 78  76  83  50]
  [ 83  85  69  51]]

 [[131 192 133 201]
  [106  34 113  64]
  [141 193 146 160]
  [122  47 106  89]]

 [[136  72 205   4]
  [138 168 217 153]
  [ 70 135  21 181]
  [167 136 226 187]]

 [[ 98  42 231 227]
  [ 95 247  46 183]
  [172  43  62 139]
  [ 85 221  63 132]]

 [[195 233  14 237]
  [ 98 149 187  12]
  [243 216 230 109]
  [ 68 153 166  34]]

 [[ 45 196 202  39]
  [ 94 203 112 124]
  [ 96 184  94  51]
  [ 17 136  46  12]]

 [[ 29 217  19  52]
  [157  86  38  90]
  [158  38 120  75]
  [221  85 123 119]]

 [[227  58  41  29]
  [ 46 120  94   4]
  [107  77  53 126]
  [197 144 235 156]]

 [[145 171 130 159]
  [221 165 251 255]
  [181 248 205 179]
  [ 97 241  26 134]]

 [[156  55 181  42]
  [176  21 238  17]
  [241   9 196 119]
  [186  75  81 215]]

 [[ 40  31 170 128]
  [ 69  80 190 175]
  [255 246  50  69]
  [ 95  20  69 146]]]
D±ég×q¨í9tjL©}"ëLh²¤}Î@Â&#ÞÊÖÄêêªÏ -ÐoG²±å0OÔ&¿\Ä:ÎîjLË¯¨ýW^·Ì¶LN  94228132298887048

Zum Entschlüsseln wenden wir erneut AES im Counter-Modus auf die ersten $n$ Blöcke an, um den Klartext zu erhalten. Vom Klartext berechnen wir erneut den Hashwert mit CBC-MAC und verifizieren, dass dieser mit dem Hashwert im letzten Block übereinstimmt.

In [18]:
def ccm_decrypt_and_verify(cryptotext: str, nonce: int, key: str = "EINSCHLUESSEL123"):
    round_keys = generate_round_keys(str_to_matrix(key))
    ctr = nonce << 64
    # Decrypt = Encrypt
    x = aes_counter(cryptotext[:-16], ctr, round_keys)
    # Verifiziere Hash
    tmp = aes_cbc_mac(x, round_keys)
    y_ = str_to_matrix(cryptotext[-16:])
    tmp_ = np.bitwise_xor(y_, int_to_matrix(ctr))
    verified = np.array_equal(tmp, tmp_)
    return x, verified

# Test (correct crypto, nonce)
dec, verified = ccm_decrypt_and_verify(enc, nonce)
print(dec, verified)
assert verified
# Test (correct crypto, incorrect nonce)
dec, verified = ccm_decrypt_and_verify(enc, nonce + 1)
print(dec, verified)
assert not verified
# Test (modified last crypto block, correct nonce)
enc_ = enc[:-16] + "MANIPULIERTERTXT"
dec, verified = ccm_decrypt_and_verify(enc_, nonce)
print(dec, verified)
assert not verified

Hallo! Wir testen heute das Counter with CBC-MAC Verfahren.      True
ßù¿5d.Kÿ]ÆóOòî`õì¦û¬ÿ¬QI	RL1CßÏÔs+ò«¹ÕLuçækk1{Iìë\ False
Hallo! Wir testen heute das Counter with CBC-MAC Verfahren.      False


Funktioniert!